## Part 2 (2 points total, 2 bonus points)

In [31]:
#open3d networkx==2.2 pyglet pyembree==0.1.5 scipy==1.2 trimesh==2.37

In [34]:
import open3d as o3d
import numpy as np
import trimesh
import os
from tqdm.notebook import tqdm as tqdm

import warnings
warnings.filterwarnings('ignore')

In [35]:
mesh_original = "data/03001627/b24ed89d85b74771216fff6094e6695c/models/model_normalized.obj"
mesh = trimesh.load_mesh(mesh_original)

In [36]:
mesh.show()

In many scenarios we want to generate a dense 3D geometry, i.e., a triangle mesh. However, from a multi-view stereo method, or a depth sensor we only obtain an unstructured point cloud. To get a triangle mesh from this unstructured input we need to perform surface reconstruction.

In [39]:
######### 1. use the open3d function read_triangle_mesh to load mesh and sample 10000 points to obtain point cloud
#mesh =
#pcd =

In [25]:
o3d.io.write_point_cloud("data/pcd_10000.ply", pcd)
print(mesh)
print(np.asarray(mesh.vertices))
print(np.asarray(mesh.triangles))

TriangleMesh with 1854 points and 618 triangles.
[[ 0.194456    0.052235    0.15345199]
 [ 0.147634    0.052235   -0.156749  ]
 [ 0.194456    0.052235   -0.156749  ]
 ...
 [-0.20939    -0.247724    0.15345199]
 [-0.162567   -0.247724    0.200275  ]
 [-0.20939    -0.247724    0.200275  ]]
[[   0    1    2]
 [   3    4    5]
 [   6    7    8]
 ...
 [1845 1846 1847]
 [1848 1849 1850]
 [1851 1852 1853]]


In [26]:

pcd = o3d.io.read_point_cloud("data/pcd_10000.ply")

In [27]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.graph_objects as go

points = np.asarray(pcd.points)

colors = None
    
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=3, color='grey')
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

In [28]:
#from open3d import JVisualizer

#visualizer = JVisualizer()
#visualizer.add_geometry(pcd)
#visualizer.show()

# Surface Reconstraction

<img src="img/PSR.jpg" width="800" height="400">

## Poisson Surface Reconstruction

Paper: http://sites.fas.harvard.edu/~cs277/papers/poissonrecon.pdf

In [29]:
######### 2. try different depth and points number in the point cloud to obtain PSR

print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    meshp, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=12)
print(meshp)
o3d.io.write_triangle_mesh("data/copy_of_mesh.ply", meshp)


run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 10000 / 10000
[Open3D DEBUG] #   Got kernel density: 0.160526 (s), 380.816 (MB) / 415.793 (MB) / 415 (MB)
[Open3D DEBUG] #     Got normal field: 0.0633211 (s), 403.246 (MB) / 415.793 (MB) / 415 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 1.751643e-04 / 1.751643e+00
[Open3D DEBUG] #       Finalized tree: 0.245539 (s), 449.027 (MB) / 449.027 (MB) / 449 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.219639 (s), 403.762 (MB) / 449.027 (MB) / 449 (MB)
[Open3D DEBUG] #Set point constraints: 0.0624869 (s), 403.762 (MB) / 449.027 (MB) / 449 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 1680869 / 179456 / 1741537
[Open3D DEBUG] Memory Usage: 403.762 MB
[Open3D DEBUG] # Linear system solved: 3.81338 (s), 408.637 (MB) / 449.027 (MB) / 449 (MB)
[Open3D DEBUG] Got average: 0.0297132 (s), 408.637 (MB) / 449.027 (MB) / 449 (MB)
[Open3D DEBUG] Iso-Value: 5.061828e-01 = 5.061828e+03 / 1.000000e+04
[Open3D DEB

True

In [30]:
import trimesh
mesh1 = trimesh.load("data/copy_of_mesh.ply")
mesh1.show()

<b>Task 2a. (2 points)</b> Process Poisson Surface reconstruction for the ShapeNet dataset item

<b>Task 2b. (2 bonus points)</b> Calculate the Chamfer distance between the original and reconstracted meshes